In [1]:
!pip install gurobipy
!pip install torch_geometric
#!pip install torch-scatter torch-sparse torch-cluster torch-spline-conv torch-geometric
!pip install scikit-optimize

from skopt import gp_minimize
from skopt.space import Real
import random

import numpy as np
import gurobipy as gp
from gurobipy import GRB
import torch
from sklearn.model_selection import train_test_split

import itertools
from sklearn.metrics import confusion_matrix, accuracy_score
from sklearn.metrics.pairwise import cosine_similarity
import seaborn as sns
import matplotlib.pyplot as plt
import os
from sklearn.metrics import pairwise_distances as sklearn_pairwise_distances
import networkx as nx
 # Use "cosine" for cosine similarity
import heapq


os.environ["GRB_LICENSE_FILE"] = "gurobi (3).lic"
os.environ["GRB_WLSACCESSID"] = "f218200d-1f8d-4342-83f5-b7b2d9263751"  # Replace with your actual WLSACCESSID
os.environ["GRB_WLSSECRET"] = "528d596b-babc-4a1e-bda2-693c44f4f006"  # Replace with your actual WLSSECRET
os.environ["GRB_LICENSEID"] = "840285"  # Replace with your actual LICENSEID

from collections import Counter
from sklearn.metrics.pairwise import cosine_distances
from torch_geometric.datasets import WebKB, WikipediaNetwork, Actor

from torch_geometric.datasets import FacebookPagePage

from torch_geometric.datasets import SNAPDataset

import numpy as np
import torch
import random
import networkx as nx
from torch_geometric.datasets import WebKB
from collections import Counter, defaultdict
from sklearn.metrics import accuracy_score
from sortedcontainers import SortedSet  # Ordered set
from torch_geometric.utils import to_networkx

from torch_geometric.datasets import WikipediaNetwork
from torch_geometric.datasets import Actor

# Load the Film dataset (also known as Actor dataset)


In [2]:

dataset = WebKB(root='data', name='Cornell')
#dataset = WebKB(root='data', name='texas')
#dataset = WikipediaNetwork(root='data', name='chameleon')
#dataset = WikipediaNetwork(root='data', name='squirrel')
#dataset = WebKB(root='data', name='Wisconsin')
#dataset = Actor(root='data/Film')
data = dataset[0]
data.edge_index = torch.cat([data.edge_index, data.edge_index.flip(0)], dim=1)

# ✅ Remove duplicate edges
data.edge_index = torch.unique(data.edge_index, dim=1)

# ✅ Convert to NetworkX Graph to Find Components
G = to_networkx(data, to_undirected=True)
components = list(nx.connected_components(G))
num_components = len(components)

print(f"\n✅ Loaded  dataset with {data.num_nodes} nodes, {data.edge_index.shape[1]} edges")
print(f"   - Features: {data.x.shape[1]}")
print(f"   - Number of Classes: {len(set(data.y.numpy()))}")
print(f"   - Number of Components: {num_components}")

# ✅ Set Training Percentage
train_percentage = 0.65

# ✅ Select Training Nodes
train_indices = []
for component in components:
    component = list(component)
    t = max(1, int(train_percentage * len(component)))
    sampled_nodes = np.random.choice(component, t, replace=False)
    train_indices.extend(sampled_nodes)

np.random.shuffle(train_indices)

# ✅ Initialize Prediction Labels
y_pred = np.full(data.num_nodes, -1)
for idx in train_indices:
    y_pred[idx] = data.y[idx].item()

# ✅ Compute Label Distribution
total_label_counts = Counter(data.y.numpy())
num_labels = len(set(data.y.numpy()))
alpha = 1  # Laplace smoothing

dataset_label_distribution = {
    lbl: (total_label_counts[lbl] + alpha) / (len(data.y) + alpha * num_labels)
    for lbl in total_label_counts.keys()
}

train_label_counts = Counter(y_pred[train_indices])

train_label_distribution = {
    lbl: (train_label_counts.get(lbl, 0) + alpha) / (len(train_indices) + alpha * num_labels)
    for lbl in dataset_label_distribution.keys()
}

# ✅ Precompute Neighbors for Each Node
A = torch.zeros((data.num_nodes, data.num_nodes))
A[data.edge_index[0], data.edge_index[1]] = 1
A = A.numpy()
neighbors_dict = {i: set(np.where(A[i] == 1)[0]) for i in range(data.num_nodes)}

# ✅ Track Instances Assigned to Each Label
class_instances = defaultdict(set)
for idx in train_indices:
    class_instances[data.y[idx].item()].add(idx)

# ✅ Ordered Set for Managing Unlabeled Vertices
ordered_set = SortedSet()
train_labeled_nodes = set(train_indices)

for node in range(data.num_nodes):
    if y_pred[node] == -1:  # Only process unlabeled nodes
        labeled_neighbors = [n for n in neighbors_dict[node] if y_pred[n] != -1]
        num_labeled_by_train = sum(1 for n in labeled_neighbors if n in train_labeled_nodes)
        num_labeled_by_propagation = len(labeled_neighbors) - num_labeled_by_train
        total_neighbors = len(neighbors_dict[node])

        if total_neighbors > 0:
            weighted_score = (num_labeled_by_propagation + 3 * num_labeled_by_train) / total_neighbors
        else:
            weighted_score = 0

        ordered_set.add((-weighted_score, node))

# ✅ Iteratively Label the Most Constrained Nodes
while ordered_set:
    _, node = ordered_set.pop(0)

    if y_pred[node] != -1:
        continue

    # Get labeled neighbors
    neighbor_labels = [y_pred[n] for n in neighbors_dict[node] if y_pred[n] != -1]
    label_counts = Counter(neighbor_labels)

    # 🚨 If this node has 0 labeled neighbors
    if not neighbor_labels:
        print(f"\n🚨 STRANGE: Node {node} was chosen, but it has 0 labeled neighbors!")
        break

    # Compute Neighbor Label Distribution
    neighbor_label_counts = Counter(neighbor_labels)
    total_labeled_neighbors = len(neighbor_labels)

    if total_labeled_neighbors > 0:
        neighbor_label_distribution = {
            lbl: neighbor_label_counts[lbl] / total_labeled_neighbors for lbl in neighbor_label_counts.keys()
        }
    else:
        neighbor_label_distribution = {lbl: 0 for lbl in dataset_label_distribution.keys()}

    # ✅ Compute Feature Distance to Each Class
    # feature_diffs = {}
    # for lbl in dataset_label_distribution.keys():
    #   if class_instances[lbl]:  # Only compute if we have instances of the label
    #       instance_features = data.x[list(class_instances[lbl])]
    #       node_feature = data.x[node].unsqueeze(0)

    #       # Compute distances for each instance
    #       distances = torch.norm(instance_features - node_feature, dim=1).tolist()

    #       # Apply weight based on whether the instance is from training or predicted labels
    #       weighted_distances = [
    #           (3 * dist) if instance in train_indices else dist  # Give 3x weight if in training set
    #           for instance, dist in zip(class_instances[lbl], distances)
    #       ]

    #       # Compute weighted average distance
    #       avg_distance = sum(weighted_distances) / len(weighted_distances)
    #       feature_diffs[lbl] = avg_distance
    #   else:
    #       feature_diffs[lbl] = 0  # No penalty if no instances exist

    feature_diffs = {}
    for lbl in dataset_label_distribution.keys():
        if class_instances[lbl]:  # Only compute if we have instances of the label
            instance_features = data.x[list(class_instances[lbl])]
            node_feature = data.x[node].unsqueeze(0)
           # print(len(instance_features),len(node_feature))
            avg_distance = torch.mean(torch.norm(instance_features - node_feature, dim=1)).item()
            feature_diffs[lbl] = avg_distance
        else:
            feature_diffs[lbl] = 0  # No penalty if no instances exist

    # ✅ Select the Best Label Based on the Adjusted Score
    best_label_candidates = []
    max_value = float('-inf')
    a1=0.5
    a2=-6
    a3=-2
    a4=1
    a5=3
    for lbl in dataset_label_distribution.keys():
        score = (
            a1*train_label_distribution.get(lbl, 0)
            +a2* neighbor_label_distribution.get(lbl, 0)
            +a3*feature_diffs[lbl]  # Penalize by average feature distance
        )
        print(lbl,score,)

        if score - max_value > 0.01:
            best_label_candidates = [lbl]
            max_value = score
        elif abs(score - max_value) <= 0.01:
            best_label_candidates.append(lbl)


    best_label = random.choice(best_label_candidates)
    y_pred[node] = best_label
    class_instances[best_label].add(node)  # Track newly labeled nodes
    neighbor_labels = [y_pred[n] for n in neighbors_dict[node] if y_pred[n] != -1]
    label_counts = Counter(neighbor_labels)

    print(f"\n🔹 Labelling Node {node} ")
    # ✅ Print Prediction Outcome
    true_label = data.y[node].item()
    correct = "✅" if best_label == true_label else "❌"
    print(f"   -> Predicted Label: {best_label} | True Label: {true_label} {correct}")

   # print(f"   - Total Neighbors: {len(neighbors_dict[node])}")
   # print(f"   - Labeled Neighbors: {len(neighbor_labels)}")
    for lbl, count in label_counts.items():
        print(f"   - Class {lbl}: {count} neighbors")
        # ✅ Update Labeled Neighbor Score for Unlabeled Neighbors
        for neighbor in neighbors_dict[node]:
            if y_pred[neighbor] == -1:
                old_entry = next((entry for entry in ordered_set if entry[1] == neighbor), None)

                if old_entry:
                    ordered_set.discard(old_entry)

                num_labeled_by_train = sum(1 for n in neighbors_dict[neighbor] if y_pred[n] != -1 and n in train_labeled_nodes)
                num_labeled_by_propagation = sum(1 for n in neighbors_dict[neighbor] if y_pred[n] != -1 and n not in train_labeled_nodes)
                total_neighbors = len(neighbors_dict[neighbor])

                new_weighted_score = (a4*num_labeled_by_propagation + a5 * num_labeled_by_train) / total_neighbors if total_neighbors > 0 else 0

                ordered_set.add((-new_weighted_score, neighbor))
    print("\n\n\n")

# ✅ Evaluate Accuracy
test_indices = [i for i in range(data.num_nodes) if i not in train_indices]
y_true = data.y[test_indices]
y_pred_test = y_pred[test_indices]

valid_idx = [i for i in range(len(y_pred_test)) if y_pred_test[i] != -1]
if valid_idx:
    final_accuracy = accuracy_score(y_true[valid_idx], y_pred_test[valid_idx])
    print(f"\n🎯 Label Propagation Accuracy: {final_accuracy:.4f}")
else:
    print("\n❌ No valid predictions were made!")



✅ Loaded  dataset with 183 nodes, 557 edges
   - Features: 1703
   - Number of Classes: 5
   - Number of Components: 1
3 -28.113268162176862
0 -29.734613325537705
4 -29.814894156727362
1 -28.090970962028194
2 -29.773610494970306

🔹 Labelling Node 6 
   -> Predicted Label: 1 | True Label: 4 ❌
   - Class 3: 1 neighbors
   - Class 4: 2 neighbors
   - Class 0: 1 neighbors




3 -21.21612384454991
0 -18.3466853165045
4 -16.714231925282053
1 -18.774593322257687
2 -21.8052553161373

🔹 Labelling Node 9 
   -> Predicted Label: 4 | True Label: 3 ❌
   - Class 3: 1 neighbors




3 -16.693592335150495
0 -26.313476469458603
4 -18.362991767201
1 -19.794204680900265
2 -22.368064306615814

🔹 Labelling Node 14 
   -> Predicted Label: 3 | True Label: 3 ✅
   - Class 0: 1 neighbors




3 -32.91293170587803
0 -33.81783285373595
4 -32.874012427601386
1 -33.51581188915222
2 -40.47799815976523

🔹 Labelling Node 19 
   -> Predicted Label: 4 | True Label: 4 ✅
   - Class 2: 1 neighbors




3 -18.977947117254985


In [3]:
import torch
import numpy as np
import random
from collections import Counter, defaultdict
from sortedcontainers import SortedSet
from sklearn.metrics import accuracy_score
import networkx as nx
from torch_geometric.utils import to_networkx

def predictclass(data, train_indices, test_indices, a1, a2, a3, a4, a5):
    data.edge_index = torch.cat([data.edge_index, data.edge_index.flip(0)], dim=1)
    train_indices = list(train_indices)
    test_indices = list(test_indices)

    # ✅ Remove duplicate edges
    data.edge_index = torch.unique(data.edge_index, dim=1)

    # ✅ Initialize Prediction Labels
    y_pred = np.full(data.num_nodes, -1, dtype=int)
    for idx in train_indices:
        y_pred[idx] = data.y[idx].item()

    # ✅ Compute Label Distribution
    total_label_counts = Counter(data.y.numpy())
    num_labels = len(set(data.y.numpy()))
    alpha = 1  # Laplace smoothing

    dataset_label_distribution = {
        lbl: (total_label_counts[lbl] + alpha) / (len(data.y) + alpha * num_labels)
        for lbl in total_label_counts.keys()
    }

    train_label_counts = Counter(y_pred[train_indices])

    train_label_distribution = {
        lbl: (train_label_counts.get(lbl, 0) + alpha) / (len(train_indices) + alpha * num_labels)
        for lbl in dataset_label_distribution.keys()
    }

    # ✅ Precompute Neighbors for Each Node
    A = torch.zeros((data.num_nodes, data.num_nodes))
    A[data.edge_index[0], data.edge_index[1]] = 1
    neighbors_dict = {i: set(np.where(A[i] == 1)[0]) for i in range(data.num_nodes)}

    # ✅ Track Instances Assigned to Each Label
    class_instances = defaultdict(set)
    for idx in train_indices:
        class_instances[data.y[idx].item()].add(idx)

    # ✅ Ordered Set for Managing Unlabeled Vertices
    ordered_set = SortedSet()
    train_labeled_nodes = set(train_indices)

    for node in range(data.num_nodes):
        if y_pred[node] == -1:
            labeled_neighbors = [n for n in neighbors_dict[node] if y_pred[n] != -1]
            num_labeled_by_train = sum(1 for n in labeled_neighbors if n in train_labeled_nodes)
            num_labeled_by_propagation = len(labeled_neighbors) - num_labeled_by_train
            total_neighbors = len(neighbors_dict[node])

            if total_neighbors > 0:
                weighted_score = (a4 * num_labeled_by_propagation + a5 * num_labeled_by_train) / total_neighbors
            else:
                weighted_score = 0

            ordered_set.add((-weighted_score, node))

    # ✅ Iteratively Label the Most Constrained Nodes
    while ordered_set:
        _, node = ordered_set.pop(0)

        if y_pred[node] != -1:
            continue  # Skip already labeled nodes

        # Get labeled neighbors
        neighbor_labels = [y_pred[n] for n in neighbors_dict[node] if y_pred[n] != -1]
        label_counts = Counter(neighbor_labels)

        # 🚨 If this node has 0 labeled neighbors
        if not neighbor_labels:
            continue

        # Compute Neighbor Label Distribution
        neighbor_label_counts = Counter(neighbor_labels)
        total_labeled_neighbors = len(neighbor_labels)

        if total_labeled_neighbors > 0:
            neighbor_label_distribution = {
                lbl: neighbor_label_counts[lbl] / total_labeled_neighbors for lbl in neighbor_label_counts.keys()
            }
        else:
            neighbor_label_distribution = {lbl: 0 for lbl in dataset_label_distribution.keys()}

        feature_diffs = {}

        for lbl in dataset_label_distribution.keys():
            if class_instances[lbl]:  # Only compute if we have instances of the label
                instance_features = data.x[list(class_instances[lbl])]
                node_feature = data.x[node].unsqueeze(0)

                if instance_features.numel() > 0:  # ✅ Ensure instance_features is not empty
                    avg_distance = torch.mean(torch.norm(instance_features - node_feature, dim=1)).item()
                else:
                    avg_distance = 0  # ✅ Avoid empty slice issue
                feature_diffs[lbl] = avg_distance
            else:
                feature_diffs[lbl] = 0  # No penalty if no instances exist

        # ✅ Select the Best Label Based on the Adjusted Score
        best_label_candidates = []
        max_value = float('-inf')

        for lbl in dataset_label_distribution.keys():
            score = (
                a1 * train_label_distribution.get(lbl, 0)
                + a2 * neighbor_label_distribution.get(lbl, 0)
                + a3 * feature_diffs[lbl]  # Penalize by average feature distance
            )

            if score - max_value > 0.01:
                best_label_candidates = [lbl]
                max_value = score
            elif abs(score - max_value) <= 0.01:
                best_label_candidates.append(lbl)

        best_label = random.choice(best_label_candidates)
        y_pred[node] = best_label
        class_instances[best_label].add(node)  # Track newly labeled nodes

        # ✅ Update Labeled Neighbor Score for Unlabeled Neighbors
        for neighbor in neighbors_dict[node]:
            if y_pred[neighbor] == -1:
                old_entry = next((entry for entry in ordered_set if entry[1] == neighbor), None)

                if old_entry:
                    ordered_set.discard(old_entry)

                num_labeled_by_train = sum(1 for n in neighbors_dict[neighbor] if y_pred[n] != -1 and n in train_labeled_nodes)
                num_labeled_by_propagation = sum(1 for n in neighbors_dict[neighbor] if y_pred[n] != -1 and n not in train_labeled_nodes)
                total_neighbors = len(neighbors_dict[neighbor])

                new_weighted_score = (a4 * num_labeled_by_propagation + a5 * num_labeled_by_train) / total_neighbors if total_neighbors > 0 else 0

                ordered_set.add((-new_weighted_score, neighbor))

    # ✅ Convert to PyTorch Tensor for Neural Network
    y_pred_test = torch.tensor(y_pred[test_indices], dtype=torch.float)

    # ✅ Compute Initial Classification Accuracy
    y_true = data.y[test_indices]
    valid_idx = [i for i in range(len(y_pred_test)) if y_pred_test[i] != -1]
    if valid_idx:
        initial_accuracy = accuracy_score(y_true[valid_idx], y_pred_test[valid_idx])
    else:
        initial_accuracy = 0.0  # No valid predictions

    return y_pred_test, initial_accuracy



In [4]:

data = dataset[0]



# Count occurrences of each class label
class_counts = torch.bincount(data.y)

# Print the number of elements in each class
for class_id, count in enumerate(class_counts):
    print(f"Class {class_id}: {count} elements")
a1=0.5
a2=-6
a3=-2
a4=1
a5=3

indices = np.arange(data.num_nodes)  # NumPy array of indices [0, 1, 2, ..., num_nodes-1]
np.random.shuffle(indices)
train_idx, test_idx = train_test_split(indices, test_size=0.8)

# Further split train_idx into train and validation sets (e.g., 20% of train goes to validation)
train_idx, val_idx = train_test_split(train_idx, test_size=0.75)
print(predictclass(data,train_idx,valid_idx,a1,a2,a3,a4,a5))

Class 0: 38 elements
Class 1: 16 elements
Class 2: 30 elements
Class 3: 82 elements
Class 4: 17 elements
(tensor([1., 1., 4., 3., 3., 4., 3., 1., 3., 4., 1., 1., 4., 4., 1., 1., 1., 1.,
        1., 3., 1., 3., 1., 4., 1., 4., 1., 2., 4., 1., 1., 1., 1., 1., 1., 1.,
        1., 1., 4., 1., 1., 1., 4., 3., 4., 1., 4., 3., 1., 1., 3., 4., 4., 1.,
        4., 1., 4., 4., 1., 1., 1., 1., 1., 1., 4.]), 0.2)


In [5]:
from sklearn.model_selection import train_test_split
from scipy.optimize import minimize
import warnings
warnings.simplefilter("error", RuntimeWarning)  # Convert warnings to errors
#dataset = WebKB(root='data', name='Cornell')
#dataset = WebKB(root='data', name='texas')
#dataset = WikipediaNetwork(root='data', name='chameleon')
#dataset = WikipediaNetwork(root='data', name='squirrel')
#dataset = WebKB(root='data', name='Wisconsin')


data = dataset[0]

num_nodes = data.x.shape[0]
indices = np.arange(num_nodes)

# Split into training (70%) and testing (30%) randomly
from sklearn.model_selection import train_test_split
random_seed = 42
train_idx, test_idx = train_test_split(indices, test_size=0.3, random_state=random_seed)
train_idx, val_idx = train_test_split(train_idx, test_size=0.3, random_state=random_seed)


# Function to optimize (negate accuracy since we minimize)
# Function to optimize (maximize train accuracy by minimizing negative accuracy)

def random_search(model,data, train_idx,val_idx, num_samples=400):
    best_params = None
    best_score = float('-inf')

    for _ in range(num_samples):
        a1 = random.uniform(0, 10)     # a1 must be positive
        a2 = random.uniform(-10, -0.01)  # a2 must be negative
        a3 = random.uniform(-10, -0.01)  # a3 must be negative
        a5 = random.uniform(0, 10)     # a5 must be positive
        a4 = random.uniform(0,a5 )    # a4 must be positive and a4 <= a5

        params = [a1, a2, a3, a4, a5]


        _,score = model(data,train_idx,val_idx,params[0],params[1],params[2],params[3],params[4])
        if score > best_score:
            best_score = score
            best_params = params

    return best_params, best_score



# Initial guesses for a1, a2, a3, a4, a5
#initial_params = [0.5,-5,-2,1,3]

# Bounds for parameters
 # Adjust bounds as needed

# Optimize using scipy
result,best_val = random_search(predictclass,data,train_idx,val_idx,num_samples=20)
#print(result)
# Get best parameters
best_a1, best_a2, best_a3, best_a4, best_a5 = result[0],result[1],result[2],result[3],result[4]

# Evaluate on test set


# Print results
print("best validation by random search:", best_val)


best validation by random search: 0.6923076923076923


In [6]:


def bayesian_optimization(model, data, train_idx, val_idx,  initial_params,num_calls=50):
    """
    Bayesian Optimization to find best hyperparameters (a1, a2, a3, a4, a5)
    that maximize the validation accuracy.

    Arguments:
        model: Function that takes (data, train_idx, val_idx, a1, a2, a3, a4, a5) and returns (loss, accuracy)
        data: Dataset object
        train_idx: Training indices
        val_idx: Validation indices
        num_calls: Number of function evaluations (default: 50)
        initial_params: Optional initial solution [a1, a2, a3, a4, a5] (default: None)

    Returns:
        best_params: The best found values for [a1, a2, a3, a4, a5]
        best_accuracy: The best validation accuracy found
    """

    # Define the search space for a1, a2, a3, a4, a5
    search_space = [
        Real(0, 10, name="a1"),        # a1 must be positive
        Real(-10, -0.01, name="a2"),   # a2 must be negative
        Real(-10, -0.01, name="a3"),   # a3 must be negative
        Real(0, 10, name="a5"),        # a5 must be positive
        Real(0, 10, name="a4"),        # a4 must be positive and a4 ≤ a5
    ]

    # Define the objective function for minimization
    def objective(params):
        a1, a2, a3, a5, a4 = params  # Bayesian optimization orders variables alphabetically
        a4 = min(a4, a5)  # Ensure a4 ≤ a5

        _, score = model(data, train_idx, val_idx, a1, a2, a3, a4, a5)
        return -score  # Since `gp_minimize` minimizes the function, we negate accuracy

    # Run Bayesian Optimization with an initial solution if provided
    if initial_params is not None:
        initial_params[3] = max(initial_params[3], initial_params[4])  # Ensure a4 ≤ a5
        result = gp_minimize(objective, search_space, n_calls=num_calls, x0=initial_params, random_state=None)
    else:
        result = gp_minimize(objective, search_space, n_calls=num_calls, random_state=None)

    # Extract best parameters
    best_a1, best_a2, best_a3, best_a5, best_a4 = result.x
    best_a4 = min(best_a4, best_a5)  # Ensure a4 ≤ a5
    best_accuracy = -result.fun

    best_params = [best_a1, best_a2, best_a3, best_a4, best_a5]

    return best_params, best_accuracy

# Example Usage:
best_params, best_validation_score = bayesian_optimization(predictclass, data, train_idx, val_idx,result)
print("best_validation=",best_validation_score)
if best_validation_score<best_val:
  print("should not happen!!")
best_a1, best_a2, best_a3, best_a4, best_a5 = best_params

# # # # Step 2: Test the Model Using the Best Parameters
y_pred_test, final_test_accuracy = predictclass(data, train_idx, test_idx, best_a1, best_a2, best_a3, best_a4, best_a5)

# # # Step 3: Print the Final Accuracy on the Test Set
print("\n🎯 Final Test Set Accuracy:", final_test_accuracy)


best_validation= 0.6923076923076923

🎯 Final Test Set Accuracy: 0.4909090909090909


In [7]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch_geometric.nn as pyg_nn
from torch_geometric.utils import to_dense_adj
from sklearn.metrics import accuracy_score
from torch.nn import Linear, ReLU, Dropout, Sequential
from torch_geometric.nn.conv import APPNP
from torch_geometric.nn import MessagePassing
from torch_geometric.nn.conv.gcn_conv import gcn_norm
from torch_geometric.utils import add_remaining_self_loops
from torch_geometric.typing import Adj, OptTensor
from torch_geometric.nn.inits import zeros

class HighPassConv(MessagePassing):
    def __init__(self, in_channels, out_channels, amp=0.5, **kwargs):
        kwargs.setdefault('aggr', 'add')
        super().__init__(**kwargs)
        self.amp = amp
        self.lin = Linear(in_channels, out_channels)
        self.bias = nn.Parameter(torch.Tensor(out_channels))
        self.reset_parameters()

    def reset_parameters(self):
        self.lin.reset_parameters()
        zeros(self.bias)

    def forward(self, x, edge_index, edge_weight=None):
        x = self.lin(x)
        out = self.propagate(edge_index, x=x, edge_weight=edge_weight)
        out = self.amp * x - out
        return out

    def message(self, x_j, edge_weight):
        return x_j if edge_weight is None else edge_weight.view(-1, 1) * x_j

class Augmenter(nn.Module):
    def __init__(self, in_dim, hidden_dim, amp=0.5):
        super().__init__()
        self.conv1 = HighPassConv(in_dim, hidden_dim, amp=amp)
        self.conv2 = HighPassConv(hidden_dim, hidden_dim, amp=amp)
        self.mlp_edge_model = Sequential(
            Dropout(0.5),
            Linear(hidden_dim, hidden_dim * 2),
            ReLU(),
            Dropout(0.5),
            Linear(hidden_dim * 2, 1)
        )

    def forward(self, x, edge_index):
        x = F.elu(self.conv1(x, edge_index))
        x = self.conv2(x, edge_index)
        src, dst = edge_index[0], edge_index[1]
        edge_emb = x[src] + x[dst]
        return torch.sigmoid(self.mlp_edge_model(edge_emb))

class HeterophilicNodeClassifier(nn.Module):
    def __init__(self, in_dim, hidden_dim, num_classes):
        super().__init__()
        self.augmenter = Augmenter(in_dim, hidden_dim)
        self.gnn_high = HighPassConv(in_dim, hidden_dim)
        self.offset_mlp = Sequential(
            Linear(in_dim, hidden_dim),
            ReLU(),
            Linear(hidden_dim, hidden_dim)
        )
        self.final_mlp = Sequential(
            Linear(hidden_dim, hidden_dim),
            ReLU(),
            Linear(hidden_dim, num_classes)
        )

    def forward(self, x, edge_index, train_indices, test_indices, initial_test_preds=None):
      #  print("\n🔄 Forward pass started")
        edge_weights = self.augmenter(x, edge_index)
        h = self.gnn_high(x, edge_index, edge_weights)
        offset = self.offset_mlp(x)
        logits = self.final_mlp(h + offset)
        expanded_preds = torch.zeros_like(logits)

        if initial_test_preds is not None:
            if initial_test_preds.shape[0] > len(test_indices):
                initial_test_preds = initial_test_preds[:len(test_indices)]
            one_hot_preds = F.one_hot(initial_test_preds.long(), num_classes=logits.shape[1]).float()
            expanded_preds[test_indices] = one_hot_preds
            logits = logits + 0.1 * (logits - expanded_preds)

        return F.log_softmax(logits, dim=1)

# Training and Evaluation Code
def train_and_evaluate(model, data, train_indices, test_indices, initial_preds, epochs=2000, lr=0.01):
    criterion = nn.NLLLoss()
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)

    for epoch in range(epochs):
        model.train()
        optimizer.zero_grad()
        logits = model(data.x, data.edge_index, train_indices, test_indices, initial_preds)
        loss = criterion(logits[train_indices], data.y[train_indices])
        loss.backward()
        optimizer.step()

    model.eval()
    with torch.no_grad():
        logits = model(data.x, data.edge_index, train_indices, test_indices, initial_preds)
        predicted_classes = torch.argmax(logits, dim=1)
        test_predictions = predicted_classes[test_indices]
        final_accuracy = accuracy_score(data.y[test_indices].cpu().numpy(), test_predictions.cpu().numpy())

   # print(f"\n🎯 Final Test Accuracy: {final_accuracy:.4f}")
    return final_accuracy, test_predictions

In [8]:
# Compute initial test predictions
initial_preds, initial_accuracy = predictclass(data, train_idx, test_idx, best_a1, best_a2, best_a3, best_a4, best_a5)

# Ensure `initial_preds` is a tensor and on the correct device
initial_preds = torch.tensor(initial_preds, dtype=torch.long, device=data.x.device)

print(f"📊 Initial Classification Accuracy: {initial_accuracy:.4f}")

# Create the model
model = HeterophilicNodeClassifier(
    in_dim=data.x.shape[1],
    hidden_dim=32,
    num_classes=len(set(data.y.tolist()))
).to(data.x.device)  # Ensure model is on the correct device

# 🔄 Train the model before inference
final_accuracy, test_predictions = train_and_evaluate(
    model, data, train_idx, test_idx, initial_preds, epochs=100, lr=0.01
)

# ✅ Model evaluation after training
model.eval()
with torch.no_grad():
    logits = model(data.x, data.edge_index, train_idx, test_idx, initial_test_preds=initial_preds)
    predicted_classes = torch.argmax(logits, dim=1)

print(f"🎯 Final Classification Accuracy: {final_accuracy:.4f}")




📊 Initial Classification Accuracy: 0.4909


<ipython-input-8-0b9ea04df471>:5: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  initial_preds = torch.tensor(initial_preds, dtype=torch.long, device=data.x.device)


🎯 Final Classification Accuracy: 0.6545
